In [ ]:
import pandas as pd
import re
import plotly.express as px
from shapely import affinity
import geopandas as gpd
import string
import nltk
from nltk.corpus import stopwords
import spacy
from collections import Counter

In [ ]:
def eliminar_urls(texto):
    """Elimina URLs de un texto dado."""
    if pd.isnull(texto):
        return texto
    return re.sub(r'https?://\S+|www\.\S+', '', texto)

def clasificar_edad_rango(edad):
    """Clasifica una edad numérica en rangos predefinidos."""
    if pd.isna(edad):
        return None
    edad = int(edad)
    if edad < 30:
        return "<30"
    elif edad < 40:
        return "30-39"
    elif edad < 50:
        return "40-49"
    elif edad < 60:
        return "50-59"
    elif edad < 70:
        return "60-69"
    return "70+"

def obtener_quinquenios(legislaturas_str):
    """
    Convierte una cadena de años separados por comas en un conjunto de rangos quinquenales.
    Ejemplo: "2000, 2003" → "2000-2004, 2000-2004"
    """
    años = {int(x.strip()) for x in legislaturas_str.split(",") if x.strip().isdigit()}
    quinquenios = {
        f"{1975 + 5 * ((año - 1975) // 5)}-{1975 + 5 * ((año - 1975) // 5) + 4}"
        for año in años
    }
    return ", ".join(sorted(quinquenios))

def agrupar_seguidores(seguidores):
    if seguidores < 1_000:
        return "Menos de 1k"
    elif seguidores < 10_000:
        return "1k - 10k"
    elif seguidores < 100_000:
        return "10k - 100k"
    elif seguidores < 1_000_000:
        return "100k - 1M"
    return "Más de 1M"

def agrupar_posts(posts):
    if posts < 1_000:
        return "Menos de 1k"
    elif posts < 10_000:
        return "1k - 10k"
    elif posts < 50_000:
        return "10k - 50k"
    elif posts < 100_000:
        return "50k - 100k"
    return "Más de 100k"

def agrupar_fechas(fechas):
    if fechas < 2010:
        return "Antes de 2010"
    elif fechas < 2014:
        return "2010 - 2014"
    elif fechas < 2020:
        return "2014 - 2019"
    elif fechas < 2025:
        return "2020 - 2024"
    return "2025 y después"

colores_partidos = {
    "PSOE": "#ff0000",
    "PP": "#189ad3",
    "SUMAR": "#ff0065",
    "VOX": "#74d600",
    "JxCAT-JUNTS": "#43e8d8",
    "EAJ-PNV": "#389844",
    "ERC": "#fdb73e",
    "EH Bildu": "#3fa0a3",
    "CCa": "#fffff2",
    "PRC": "#d6ff00",
    "BNG": "#b0cfff",
    "Más Madrid": "#52eb86",
    "UPN": "#0059b3",
}

In [ ]:
ruta_excel = "clasificador_analisis/analisis/datasets/politicos_etiquetado_actualizado.xlsx"

df_metadata    = pd.read_excel(ruta_excel, sheet_name="Metadata")
df_posts       = pd.read_excel(ruta_excel, sheet_name="Posts")
df_comentarios = pd.read_excel(ruta_excel, sheet_name="Comentarios")

## Calcular índices de popularidad e interacción de políticos y partidos en Twitter

### Número de seguidores por político

In [ ]:
top10_popularidad = (
    df_metadata[["Nombre", "Partido", "Seguidores"]]
    .nlargest(10, "Seguidores")  
)

orden_invertido = top10_popularidad["Nombre"]

fig_top10 = px.bar(
    top10_popularidad,
    x="Nombre",
    y="Seguidores",
    color="Partido",
    text="Seguidores",
    title="Top 10 políticos más populares en Twitter (por número de seguidores)",
    color_discrete_map=colores_partidos
)

fig_top10.update_traces(textposition="outside")

fig_top10.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Nombre del político",
    yaxis_title="Número de seguidores",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig_top10.show()


In [ ]:
top10_posts = (
    df_metadata[["Nombre", "Partido", "Posts"]]
    .nlargest(10, "Posts")  
)

orden_invertido = top10_posts["Nombre"]

fig_top10 = px.bar(
    top10_posts,
    x="Nombre",
    y="Posts",
    color="Partido",
    text="Posts",
    title="Top 10 políticos con mayor número de posts en Twitter",
    color_discrete_map=colores_partidos
)

fig_top10.update_traces(textposition="outside")

fig_top10.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Nombre del político",
    yaxis_title="Número de seguidores",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig_top10.show()


### Número de seguidores por partido

In [ ]:
top10_partidos = (
    df_metadata.groupby("Partido", as_index=False)["Seguidores"]
    .sum()
    .nlargest(10, "Seguidores")
)

orden_invertido = top10_partidos["Partido"]

fig_partidos = px.bar(
    top10_partidos,
    x="Partido",
    y="Seguidores",
    text="Seguidores",
    color="Partido",
    title="Top 10 partidos más populares en Twitter (por número total de seguidores)",
    color_discrete_map=colores_partidos
)

fig_partidos.update_traces(textposition="outside")

fig_partidos.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Partido político",
    yaxis_title="Total de seguidores",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=80)
)

fig_partidos.show()


In [ ]:
top10_partidos = (
    df_metadata.groupby("Partido", as_index=False)["Posts"]
    .sum()
    .nlargest(10, "Posts")
)

orden_invertido = top10_partidos["Partido"]

fig_partidos = px.bar(
    top10_partidos,
    x="Partido",
    y="Posts",
    text="Posts",
    color="Partido",
    title="Top 10 partidos con mayor número de posts en Twitter",
    color_discrete_map=colores_partidos
)

fig_partidos.update_traces(textposition="outside")

fig_partidos.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Partido político",
    yaxis_title="Total de seguidores",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=80)
)

fig_partidos.show()


### Tasa de posts por año por político

In [ ]:
año_actual = 2025

df_metadata["Tasa_Posts_Año"] = df_metadata.apply(
    lambda row: round(row["Posts"] / (año_actual - row["Comienzo en X/Twitter"]), 0)
    if row["Comienzo en X/Twitter"] and (año_actual - row["Comienzo en X/Twitter"]) > 0 else 0,
    axis=1
)

top10_post_rate = (
    df_metadata[["Nombre", "Partido", "Tasa_Posts_Año"]]
    .nlargest(10, "Tasa_Posts_Año")
)

orden_invertido = top10_post_rate["Nombre"]

fig_post_rate = px.bar(
    top10_post_rate,
    x="Nombre",
    y="Tasa_Posts_Año",
    color="Partido",
    text="Tasa_Posts_Año",
    title="Top 10 políticos con mayor tasa de publicación anual en Twitter",
    color_discrete_map=colores_partidos
)

fig_post_rate.update_traces(textposition="outside")

fig_post_rate.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Nombre del político",
    yaxis_title="Posts por año",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig_post_rate.show()


In [ ]:
tasa_post_partido = (
    df_metadata.groupby("Partido")["Tasa_Posts_Año"]
    .mean()
    .round(0)
    .sort_values(ascending=False)
    .reset_index()
)

top10_tasa_post_partido = tasa_post_partido.head(10)

orden_invertido = top10_tasa_post_partido["Partido"]
fig_post_rate_partido = px.bar(
    top10_tasa_post_partido,
    x="Partido",
    y="Tasa_Posts_Año",
    text="Tasa_Posts_Año",
    color="Partido",
    title="Top 10 partidos con mayor tasa media de publicación anual",
    color_discrete_map=colores_partidos
)

fig_post_rate_partido.update_traces(textposition="outside")

fig_post_rate_partido.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Partido",
    yaxis_title="Posts por año (promedio)",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig_post_rate_partido.show()


### Tasa de seguidores por año por político

In [ ]:
df_metadata["Tasa_Seguidores_Año"] = df_metadata.apply(
    lambda row: round(row["Seguidores"] / (año_actual - row["Comienzo en X/Twitter"]), 0)
    if row["Comienzo en X/Twitter"] and (año_actual - row["Comienzo en X/Twitter"]) > 0 else 0,
    axis=1
)


In [ ]:
top10_seguidores_rate = (
    df_metadata[["Nombre", "Partido", "Tasa_Seguidores_Año"]]
    .nlargest(10, "Tasa_Seguidores_Año")
)

orden_invertido = top10_seguidores_rate["Nombre"]

fig_seguidores_rate = px.bar(
    top10_seguidores_rate,
    x="Nombre",
    y="Tasa_Seguidores_Año",
    color="Partido",
    text="Tasa_Seguidores_Año",
    title="Top 10 políticos con mayor tasa de seguidores anual en Twitter",
    color_discrete_map=colores_partidos
)

fig_seguidores_rate.update_traces(textposition="outside")

fig_seguidores_rate.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Nombre del político",
    yaxis_title="Seguidores por año",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig_seguidores_rate.show()


In [ ]:
tasa_seguidores_partido = (
    df_metadata.groupby("Partido")["Tasa_Seguidores_Año"]
    .mean()
    .round(0)
    .sort_values(ascending=False)
    .reset_index()
)

top10_tasa_seguidores_partido = tasa_seguidores_partido.head(10)
orden_invertido = top10_tasa_seguidores_partido["Partido"]

fig_seguidores_rate_partido = px.bar(
    top10_tasa_seguidores_partido,
    x="Partido",
    y="Tasa_Seguidores_Año",
    text="Tasa_Seguidores_Año",
    color="Partido",
    title="Top 10 partidos con mayor tasa media de seguidores anual",
    color_discrete_map=colores_partidos
)

fig_seguidores_rate_partido.update_traces(textposition="outside")

fig_seguidores_rate_partido.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Partido",
    yaxis_title="Seguidores por año (promedio)",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig_seguidores_rate_partido.show()


### Índice de interacción promedio (Likes + Retweets + Comenatrios / Posts)

In [ ]:
interacciones = (
    df_posts.groupby("ID_Político")
    .agg({
        "Likes": "sum",
        "Retweets": "sum",
        "Comentarios_Totales": "sum",
        "Enlace_Post": "count"  
    })
    .rename(columns={"Enlace_Post": "Posts_extraidos"})
    .reset_index()
)

interacciones["Interacción"] = round(
    (interacciones["Likes"] + interacciones["Retweets"] + interacciones["Comentarios_Totales"]) / interacciones["Posts_extraidos"],
    0
)

df_metadata = df_metadata.merge(interacciones, on="ID_Político", how="left")


In [ ]:
top10_interaccion = df_metadata.nlargest(10, "Interacción")

orden_invertido = top10_interaccion["Nombre"]

fig = px.bar(
    top10_interaccion,
    x="Nombre",
    y="Interacción",
    color="Partido",
    text="Interacción",
    title="Top 10 políticos con mayor interacción promedio por publicación",
    color_discrete_map=colores_partidos
)

fig.update_traces(textposition="outside")

fig.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Nombre del político",
    yaxis_title="Interacción promedio",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=80)
)

fig.show()


In [ ]:
top10_partidos_interaccion = (
    df_metadata.groupby("Partido", as_index=False)["Interacción"]
    .mean()
    .nlargest(10, "Interacción")
    .round(0)
)


In [ ]:
orden_invertido = top10_partidos_interaccion["Partido"]

fig = px.bar(
    top10_partidos_interaccion,
    x="Partido",
    y="Interacción",
    text="Interacción",
    color="Partido",
    title="Top 10 partidos con mayor interacción promedio por publicación",
    color_discrete_map=colores_partidos
)

fig.update_traces(textposition="outside")

fig.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Partido político",
    yaxis_title="Interacción promedio",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=80)
)

fig.show()


### Interacción relativa (Interacción promedio / Número de seguidores)

In [ ]:
df_metadata["Interacción_Relativa"] = df_metadata.apply(
    lambda row: row["Interacción"] / row["Seguidores"] if row["Seguidores"] > 0 else 0,
    axis=1
)

top10_relativa = df_metadata.nlargest(10, "Interacción_Relativa").copy()
top10_relativa["Interacción_Relativa"] = top10_relativa["Interacción_Relativa"].round(3)


In [ ]:
orden_invertido = top10_relativa["Nombre"]

fig = px.bar(
    top10_relativa,
    x="Nombre",
    y="Interacción_Relativa",
    text="Interacción_Relativa",
    color="Partido",
    title="Top 10 políticos con mayor interacción relativa (por seguidor)",
    color_discrete_map=colores_partidos
)

fig.update_traces(textposition="outside")

fig.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Nombre del político",
    yaxis_title="Interacción relativa",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=100)
)

fig.show()


In [ ]:
top10_interaccion_relativa = (
    df_metadata.groupby("Partido", as_index=False)["Interacción_Relativa"]
    .mean()
    .nlargest(10, "Interacción_Relativa")
    .copy()
)

top10_interaccion_relativa["Interacción_Relativa"] = top10_interaccion_relativa["Interacción_Relativa"].round(3)


In [ ]:
orden_invertido = top10_interaccion_relativa["Partido"]

fig = px.bar(
    top10_interaccion_relativa,
    x="Partido",
    y="Interacción_Relativa",
    text="Interacción_Relativa",
    color="Partido",
    title="Top 10 partidos con mayor interacción relativa promedio",
    color_discrete_map=colores_partidos
)

fig.update_traces(textposition="outside")

fig.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=orden_invertido),
    xaxis_title="Partido político",
    yaxis_title="Interacción relativa promedio",
    width=900,
    height=600,
    margin=dict(l=20, r=20, t=60, b=80)
)

fig.show()


## Identificar patrones de actividad regional

### Número de seguidores promedio por comunidad autónoma

In [ ]:
ruta = "C:/Users/Jaime/Desktop/TFG Modelo/clasificador_analisis/analisis/Mapas/ComunidadesAutonomas_ETRS89_30N/Comunidades_Autonomas_ETRS89_30N.shp"
gdf = gpd.read_file(ruta)

idx_canarias = gdf[gdf["Texto"] == "Canarias"].index[0]
gdf.loc[idx_canarias, "geometry"] = affinity.translate(
    gdf.loc[idx_canarias, "geometry"], xoff=550_000, yoff=750_000
)

gdf = gdf.to_crs(epsg=4326)

geojson_ccaa = gdf.__geo_interface__

popularidad_ccaa = (
    df_metadata.groupby("Comunidad Autónoma", as_index=False)["Seguidores"]
    .mean()
    .round(0)
)

popularidad_ccaa["Comunidad Autónoma"] = popularidad_ccaa["Comunidad Autónoma"].replace({
    "Castilla-La Mancha": "Castilla - La Mancha"
})

fig = px.choropleth(
    popularidad_ccaa,
    geojson=geojson_ccaa,
    locations="Comunidad Autónoma",
    featureidkey="properties.Texto",
    color="Seguidores",
    color_continuous_scale="YlOrRd",
    title="Popularidad media en Twitter por Comunidad Autónoma",
    labels={"Seguidores": "Seguidores promedio"},
    width=800,
    height=500
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin=dict(r=0, t=50, l=0, b=0))
fig.show()


### Interacción relativa por comunidad autónoma

In [ ]:
interaccion_ccaa = (
    df_metadata.groupby("Comunidad Autónoma", as_index=False)["Interacción_Relativa"]
    .mean()
    .round(3)
)

interaccion_ccaa["Comunidad Autónoma"] = interaccion_ccaa["Comunidad Autónoma"].replace({
    "Castilla-La Mancha": "Castilla - La Mancha"
})

fig = px.choropleth(
    interaccion_ccaa,
    geojson=geojson_ccaa,
    locations="Comunidad Autónoma",
    featureidkey="properties.Texto",
    color="Interacción_Relativa",
    color_continuous_scale="Blues",
    title="Interacción relativa en Twitter por Comunidad Autónoma",
    labels={"Interacción_Relativa": "Interacción / Seguidor"},
    width=900,
    height=500
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin=dict(r=0, t=50, l=0, b=0))
fig.show()


### Frecuencia media de publicación por comunidad autónoma

In [ ]:
posts_ccaa = (
    df_metadata.groupby("Comunidad Autónoma", as_index=False)["Posts"]
    .mean()
    .round(0)
)

posts_ccaa["Comunidad Autónoma"] = posts_ccaa["Comunidad Autónoma"].replace({
    "Castilla-La Mancha": "Castilla - La Mancha"
    
})

fig = px.choropleth(
    posts_ccaa,
    geojson=geojson_ccaa,
    locations="Comunidad Autónoma",
    featureidkey="properties.Texto",
    color="Posts",
    color_continuous_scale="Greens",
    title="Frecuencia media de publicación en Twitter por Comunidad Autónoma",
    labels={"Posts": "Nº medio de publicaciones"},
    width=900,
    height=500
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin=dict(r=0, t=50, l=0, b=0))
fig.show()


## Análisis del tono

### Agrupación por político

In [ ]:
tono_posts_politico = (
    df_posts.groupby(["ID_Político", "Tono"])
    .size()
    .reset_index(name="Cantidad")
)

tono_posts_politico["Proporción"] = (
    tono_posts_politico["Cantidad"] / tono_posts_politico.groupby("ID_Político")["Cantidad"].transform("sum")
)


In [ ]:
comentarios_con_politico = df_comentarios.merge(
    df_posts[["Enlace_Post", "ID_Político"]],
    on="Enlace_Post",
    how="left"
)

def calcular_proporcion(df, columna_tono):
    agrupado = (
        df.groupby(["ID_Político", columna_tono])
        .size()
        .reset_index(name="Cantidad")
    )
    agrupado["Proporción"] = agrupado["Cantidad"] / agrupado.groupby("ID_Político")["Cantidad"].transform("sum")
    return agrupado

tono_comentarios_politico = calcular_proporcion(comentarios_con_politico, "Tono")

tono_respuestas_politico = calcular_proporcion(comentarios_con_politico, "Tono_Respuesta")


### Conteo por partido

In [ ]:
tono_posts_politico = (
    df_posts.groupby(["ID_Político", "Tono"])
    .size()
    .reset_index(name="Cantidad")
)
tono_posts_politico["Proporción"] = (
    tono_posts_politico["Cantidad"] / tono_posts_politico.groupby("ID_Político")["Cantidad"].transform("sum")
)

tono_posts_politico = tono_posts_politico.merge(
    df_metadata[["ID_Político", "Partido"]],
    on="ID_Político",
    how="left"
)

proporcion_partido = (
    tono_posts_politico.groupby(["Partido", "Tono"], as_index=False)["Proporción"]
    .mean()
)


In [ ]:
def graficar_proporcion_tono(df, tono, top=10):
    df_tono = df[df["Tono"] == tono].sort_values("Proporción", ascending=False).head(top)
    orden = df_tono["Partido"]

    fig = px.bar(
        df_tono,
        x="Partido",
        y="Proporción",
        text="Proporción",
        color="Partido",
        title=f"Top {top} partidos con mayor proporción de posts {tono.lower()}",
        color_discrete_map=colores_partidos
    )

    fig.update_traces(texttemplate="%{text:.2%}", textposition="outside")

    fig.update_layout(
        xaxis=dict(categoryorder="array", categoryarray=orden),
        xaxis_title="Partido",
        yaxis_title="Proporción de posts",
        width=1000,
        height=500
    )

    fig.show()

graficar_proporcion_tono(proporcion_partido, "Positivo")
graficar_proporcion_tono(proporcion_partido, "Negativo")
graficar_proporcion_tono(proporcion_partido, "Neutro")


### Conteo por político

In [ ]:
tono_posts_politico = (
    df_posts.groupby(["ID_Político", "Tono"])
    .size()
    .reset_index(name="Cantidad")
)
tono_posts_politico["Proporción"] = tono_posts_politico.groupby("ID_Político")["Cantidad"].transform(lambda x: x / x.sum())

tono_posts_politico = tono_posts_politico.merge(
    df_metadata[["ID_Político", "Nombre", "Partido"]],
    on="ID_Político",
    how="left"
)

def graficar_proporcion_tono_politico(df, tono, top=10):
    df_tono = df[df["Tono"] == tono].sort_values("Proporción", ascending=False).head(top)
    orden = df_tono["Nombre"]

    fig = px.bar(
        df_tono,
        x="Nombre",
        y="Proporción",
        text="Proporción",
        color="Partido",
        title=f"Top {top} políticos con mayor proporción de posts {tono.lower()}",
        color_discrete_map=colores_partidos
    )

    fig.update_traces(texttemplate="%{text:.2%}", textposition="outside")

    fig.update_layout(
        xaxis=dict(categoryorder="array", categoryarray=orden),
        xaxis_title="Político",
        yaxis_title="Proporción de posts",
        width=1000,
        height=600
    )

    fig.show()

graficar_proporcion_tono_politico(tono_posts_politico, "Positivo")
graficar_proporcion_tono_politico(tono_posts_politico, "Negativo")
graficar_proporcion_tono_politico(tono_posts_politico, "Neutro")


In [ ]:
posts_con_ccaa = df_posts.merge(
    df_metadata[["ID_Político", "Comunidad Autónoma"]],
    on="ID_Político",
    how="left"
)

tono_ccaa = (
    posts_con_ccaa.groupby(["Comunidad Autónoma", "Tono"])
    .size()
    .reset_index(name="Cantidad")
)

tono_ccaa["Total"] = tono_ccaa.groupby("Comunidad Autónoma")["Cantidad"].transform("sum")
tono_ccaa["Proporción"] = tono_ccaa["Cantidad"] / tono_ccaa["Total"]

tono_ccaa["Comunidad Autónoma"] = tono_ccaa["Comunidad Autónoma"].replace({
    "Castilla-La Mancha": "Castilla - La Mancha"
    
})

def mapa_tono_ccaa(df, tono, escala="YlGnBu"):
    df_tono = df[df["Tono"] == tono]

    fig = px.choropleth(
        df_tono,
        geojson=geojson_ccaa,
        locations="Comunidad Autónoma",
        featureidkey="properties.Texto",
        color="Proporción",
        color_continuous_scale=escala,
        title=f"{tono}",
        labels={"Proporción": "Proporción"},
        width=900,
        height=500
    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})
    fig.show()

# 📍 5. Visualizar mapas por tono
mapa_tono_ccaa(tono_ccaa, "Positivo", escala="Greens")
mapa_tono_ccaa(tono_ccaa, "Negativo", escala="Reds")
mapa_tono_ccaa(tono_ccaa, "Neutro", escala="Blues")


## Creación corpus

In [ ]:
nltk.download('stopwords', quiet=True)
stopwords_es = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    """
    Limpia un texto eliminando URLs, menciones, puntuación, números y stopwords.
    Devuelve una lista de tokens significativos.
    """
    if pd.isnull(texto):
        return []

    texto = texto.lower()

    texto = re.sub(r"http\S+|www\S+|@\w+", "", texto)

    texto = texto.translate(str.maketrans('', '', string.punctuation + string.digits))

    tokens = [t for t in texto.split() if t not in stopwords_es and len(t) > 2]

    return tokens


In [ ]:
df_posts["Corpus_Tokens"] = df_posts["Contenido_Traducido"].apply(limpiar_texto)
df_comentarios["Corpus_Tokens_Comentarios"] = df_comentarios["Comentario_Traducido"].apply(limpiar_texto)
df_comentarios["Corpus_Tokens_Respuestas"] = df_comentarios["Respuesta_Traducida"].apply(limpiar_texto)

corpus_posts = [token for tokens in df_posts["Corpus_Tokens"].dropna() for token in tokens]
corpus_comentarios = [token for tokens in df_comentarios["Corpus_Tokens_Comentarios"].dropna() for token in tokens]
corpus_respuestas = [token for tokens in df_comentarios["Corpus_Tokens_Respuestas"].dropna() for token in tokens]

corpus_global = corpus_posts + corpus_comentarios + corpus_respuestas


In [ ]:
nlp = spacy.load("es_core_news_md")

def extraer_entidades(texto):
    """Extrae entidades nombradas de tipo persona, organización o lugar."""
    if pd.isnull(texto):
        return []

    doc = nlp(texto)
    return [
        ent.text.strip().replace("\n", " ")
        for ent in doc.ents
        if ent.label_ in {"PER", "LOC", "ORG"}
    ]

df_posts["Entidades"] = df_posts["Contenido_Traducido"].apply(extraer_entidades)
df_comentarios["Entidades_Comentarios"] = df_comentarios["Comentario_Traducido"].apply(extraer_entidades)
df_comentarios["Entidades_Respuestas"] = df_comentarios["Respuesta_Traducida"].apply(extraer_entidades)


## Análisis básico corpus

In [ ]:

def contar_mas_frecuentes(lista_columnas, top_n=20):
    """Aplana listas y devuelve los elementos más frecuentes."""
    elementos = [item for sublista in lista_columnas.dropna() for item in sublista]
    return Counter(elementos).most_common(top_n)

def graficar_top(counter_list, titulo):
    """Genera gráfico de barras para elementos más frecuentes."""
    df = pd.DataFrame(counter_list, columns=["Término", "Frecuencia"])
    orden = df["Término"].tolist() 
    fig = px.bar(df, x="Término", y="Frecuencia", title=titulo)
    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis=dict(categoryorder="array", categoryarray=orden),
        width=1000,
        height=500,
        margin=dict(l=20, r=20, t=60, b=120)
    )
    fig.show()

top_tokens_posts = contar_mas_frecuentes(df_posts["Corpus_Tokens"])
top_entidades_posts = contar_mas_frecuentes(df_posts["Entidades"])
top_tokens_comentarios = contar_mas_frecuentes(df_comentarios["Corpus_Tokens_Comentarios"])
top_entidades_comentarios = contar_mas_frecuentes(df_comentarios["Entidades_Comentarios"])
top_tokens_respuestas = contar_mas_frecuentes(df_comentarios["Corpus_Tokens_Respuestas"])
top_entidades_respuestas = contar_mas_frecuentes(df_comentarios["Entidades_Respuestas"])

graficar_top(top_tokens_posts, "Tokens más frecuentes en Posts")
graficar_top(top_entidades_comentarios, "Entidades más frecuentes en Comentarios")


In [ ]:
top_tokens = {
    "Posts": top_tokens_posts,
    "Comentarios": top_tokens_comentarios,
    "Respuestas": top_tokens_respuestas,
}

top_entidades = {
    "Posts": top_entidades_posts,
    "Comentarios": top_entidades_comentarios,
    "Respuestas": top_entidades_respuestas,
}

for fuente, datos in top_tokens.items():
    graficar_top(datos, f"Tokens más frecuentes en {fuente}")

for fuente, datos in top_entidades.items():
    graficar_top(datos, f"Entidades más frecuentes en {fuente}")


In [ ]:
def contar_por_tono(df, columna_tokens, columna_tono):
    tonos = df[columna_tono].dropna().unique()
    resultados = {}

    for tono in tonos:
        subset = df[df[columna_tono] == tono]
        tokens_planos = [
            token for lista in subset[columna_tokens]
            if isinstance(lista, list)
            for token in lista
        ]
        resultados[tono] = Counter(tokens_planos).most_common(20)
    
    return resultados

def graficar_por_tono(diccionario_tonos, tipo, fuente):
    for tono, counter_list in diccionario_tonos.items():
        titulo = f"{tipo} más frecuentes en {fuente} ({tono})"
        graficar_top(counter_list, titulo)

def limpiar_lista_urls(lista):
    if lista is None or not isinstance(lista, list):
        return []
    return [
        eliminar_urls(str(item)).strip()
        for item in lista
        if isinstance(item, str) and eliminar_urls(str(item)).strip()
    ]

df_posts["Corpus_Tokens"] = df_posts["Corpus_Tokens"].apply(limpiar_lista_urls)
df_posts["Entidades"] = df_posts["Entidades"].apply(limpiar_lista_urls)

df_comentarios["Corpus_Tokens_Comentarios"] = df_comentarios["Corpus_Tokens_Comentarios"].apply(limpiar_lista_urls)
df_comentarios["Corpus_Tokens_Respuestas"] = df_comentarios["Corpus_Tokens_Respuestas"].apply(limpiar_lista_urls)
df_comentarios["Entidades_Comentarios"] = df_comentarios["Entidades_Comentarios"].apply(limpiar_lista_urls)
df_comentarios["Entidades_Respuestas"] = df_comentarios["Entidades_Respuestas"].apply(limpiar_lista_urls)


In [ ]:
tokens_posts_tono = contar_por_tono(df_posts, "Corpus_Tokens", "Tono")
entidades_posts_tono = contar_por_tono(df_posts, "Entidades", "Tono")

tokens_comentarios_tono = contar_por_tono(df_comentarios, "Corpus_Tokens_Comentarios", "Tono")
entidades_comentarios_tono = contar_por_tono(df_comentarios, "Entidades_Comentarios", "Tono")

tokens_respuestas_tono = contar_por_tono(df_comentarios, "Corpus_Tokens_Respuestas", "Tono_Respuesta")
entidades_respuestas_tono = contar_por_tono(df_comentarios, "Entidades_Respuestas", "Tono_Respuesta")

graficar_por_tono(tokens_posts_tono, "Tokens", "Posts")
graficar_por_tono(tokens_comentarios_tono, "Tokens", "Comentarios")
graficar_por_tono(tokens_respuestas_tono, "Tokens", "Respuestas")

graficar_por_tono(entidades_posts_tono, "Entidades", "Posts")
graficar_por_tono(entidades_comentarios_tono, "Entidades", "Comentarios")
graficar_por_tono(entidades_respuestas_tono, "Entidades", "Respuestas")


In [ ]:
from collections import Counter

def top_elementos(df, columna, n=15):
    """Devuelve el top N elementos más frecuentes de una columna de listas (tokens o entidades)"""
    counter = Counter([elem for lista in df[columna].dropna() for elem in lista])
    return set([e for e, _ in counter.most_common(n)])

def top_elementos_por(df, columna_elementos, columna_categoria, valor_categoria, n=15):
    """Top N elementos dentro de una categoría (e.g., Tono o Tema)"""
    subset = df[df[columna_categoria] == valor_categoria]
    return top_elementos(subset, columna_elementos, n=n)

def comparar_tops_por_categoria(df, columna_elementos, columna_categoria, categorias, n=15):
    tops = {cat: top_elementos_por(df, columna_elementos, columna_categoria, cat, n) for cat in categorias}
    
    comunes = set.intersection(*tops.values())
    print(f"\n🎯 Elementos comunes entre {', '.join(categorias)}: {comunes}")

    for i, cat1 in enumerate(categorias):
        for cat2 in categorias[i+1:]:
            print(f"🎯 Comunes entre {cat1} y {cat2}: {tops[cat1] & tops[cat2]}")

    for cat in categorias:
        resto = set.union(*(tops[c] for c in categorias if c != cat))
        print(f"❗ Exclusivos de {cat}: {tops[cat] - resto}")


### Comparaciones por tipo de mensaje

In [ ]:
print("\n🔍 Comparativa de TOKENS por tipo de mensaje:")
comparar_tops_por_categoria(
    df=pd.concat([
        df_posts.assign(Tipo="Posts", Tokens=df_posts["Corpus_Tokens"]),
        df_comentarios.assign(Tipo="Comentarios", Tokens=df_comentarios["Corpus_Tokens_Comentarios"]),
        df_comentarios.assign(Tipo="Respuestas", Tokens=df_comentarios["Corpus_Tokens_Respuestas"])
    ]),
    columna_elementos="Tokens",
    columna_categoria="Tipo",
    categorias=["Posts", "Comentarios", "Respuestas"],
    n=20
)

print("\n🔍 Comparativa de ENTIDADES por tipo de mensaje:")
comparar_tops_por_categoria(
    df=pd.concat([
        df_posts.assign(Tipo="Posts", Ents=df_posts["Entidades"]),
        df_comentarios.assign(Tipo="Comentarios", Ents=df_comentarios["Entidades_Comentarios"]),
        df_comentarios.assign(Tipo="Respuestas", Ents=df_comentarios["Entidades_Respuestas"])
    ]),
    columna_elementos="Ents",
    columna_categoria="Tipo",
    categorias=["Posts", "Comentarios", "Respuestas"],
    n=20
)


### Comparaciones por tono

In [ ]:
print("\n🧠 Comparativa de TOKENS por tono:")
comparar_tops_por_categoria(
    df=df_posts.rename(columns={"Corpus_Tokens": "Tokens"}),
    columna_elementos="Tokens",
    columna_categoria="Tono",
    categorias=["Positivo", "Negativo", "Neutro"],
    n=20
)

print("\n🧠 Comparativa de ENTIDADES por tono:")
comparar_tops_por_categoria(
    df=df_posts.rename(columns={"Entidades": "Ents"}),
    columna_elementos="Ents",
    columna_categoria="Tono",
    categorias=["Positivo", "Negativo", "Neutro"],
    n=20
)


In [ ]:
def obtener_top_por_tema(df, columna_lista, columna_tema, n=20):
    """
    Devuelve un diccionario con top elementos (tokens o entidades) por tema.
    """
    temas = df[columna_tema].dropna().unique()
    resultados = {}
    
    for tema in temas:
        subset = df[df[columna_tema] == tema]
        elementos = [item for sublista in subset[columna_lista].dropna() for item in sublista if isinstance(sublista, list)]
        resultados[tema] = Counter(elementos).most_common(n)
    
    return resultados


def obtener_tono_predominante_por_tema(df, columna_tema, columna_tono):
    """
    Devuelve el tono más frecuente por tema.
    """
    return (
        df.groupby(columna_tema)[columna_tono]
        .agg(lambda x: x.value_counts().idxmax())
        .reset_index(name="Tono_predominante")
    )

top_tokens_por_tema = obtener_top_por_tema(df_posts, "Corpus_Tokens", "Tema")

top_entidades_por_tema = obtener_top_por_tema(df_posts, "Entidades", "Tema")

tono_predominante = obtener_tono_predominante_por_tema(df_posts, "Tema", "Tono")


for tema in top_tokens_por_tema:
    print(f"\n🟩 Tema: {tema}")
    print(f"🔹 Tono predominante: {tono_predominante[tono_predominante['Tema'] == tema]['Tono_predominante'].values[0]}")
    print("🔠 Palabras clave más frecuentes:")
    for token, count in top_tokens_por_tema[tema]:
        print(f"  - {token}: {count}")
    print("🏷️ Entidades más frecuentes:")
    for entidad, count in top_entidades_por_tema.get(tema, []):
        print(f"  - {entidad}: {count}")


In [ ]:
def graficar_top_por_tema(diccionario, tipo="Tokens"):
    """
    Genera una gráfica de barras para cada tema con sus términos o entidades más frecuentes.
    """
    for tema, lista in diccionario.items():
        df_tema = pd.DataFrame(lista, columns=["Término", "Frecuencia"])
        orden = df_tema["Término"].tolist()

        fig = px.bar(
            df_tema,
            x="Término",
            y="Frecuencia",
            title=f"{tipo} más frecuentes en el tema: {tema}",
        )
        fig.update_layout(
            xaxis=dict(categoryorder="array", categoryarray=orden),
            xaxis_tickangle=-45,
            width=1000,
            height=500,
            margin=dict(l=20, r=20, t=60, b=120)
        )
        fig.show()


In [ ]:
graficar_top_por_tema(top_tokens_por_tema, tipo="Tokens")
graficar_top_por_tema(top_entidades_por_tema, tipo="Entidades")


In [ ]:
temas = df_posts["Tema"].dropna().unique().tolist()
comparar_tops_por_categoria(df_posts, "Corpus_Tokens", "Tema", temas, n=20)

comparar_tops_por_categoria(df_posts, "Entidades", "Tema", temas, n=20)


In [ ]:
proporcion_tono_por_tema = (
    df_posts.groupby(["Tema", "Tono"])
    .size()
    .reset_index(name="Cantidad")
)

proporcion_tono_por_tema["Proporción"] = proporcion_tono_por_tema.groupby("Tema")["Cantidad"].transform(lambda x: x / x.sum())


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

def graficar_tono_temas_combinado(df):
    temas = df["Tema"].unique()
    orden_tonos = ["Negativo", "Neutro", "Positivo"]
    color_map = {"Positivo": "green", "Negativo": "red", "Neutro": "gray"}

    n = len(temas)
    cols = 2
    rows = math.ceil(n / cols)

    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=[f"{tema}" for tema in temas],
        specs=[[{"type": "domain"} for _ in range(cols)] for _ in range(rows)],
    )

    for i, tema in enumerate(temas):
        df_tema = df[df["Tema"] == tema]
        valores = []
        etiquetas = []
        colores = []

        for tono in orden_tonos:
            cantidad = df_tema[df_tema["Tono"] == tono]["Cantidad"].sum()
            if cantidad > 0:
                valores.append(cantidad)
                etiquetas.append(tono)
                colores.append(color_map[tono])

        insidetextcolors = []
        for tono in etiquetas:
            if tono == "Neutro":
                insidetextcolors.append("black")
            else:
                insidetextcolors.append("white")

        row = i // cols + 1
        col = i % cols + 1

        fig.add_trace(
            go.Pie(
                labels=etiquetas,
                values=valores,
                marker_colors=colores,
                textinfo="label+percent",
                hovertemplate="%{label} = %{value} publicaciones<extra></extra>",
                insidetextfont=dict(color=insidetextcolors)
            ),
            row=row,
            col=col
        )

    fig.update_layout(
        height=500 * rows,
        width=900,
        showlegend=False
    )

    for annotation in fig.layout.annotations:
        annotation.y += 0.03

    fig.show()

graficar_tono_temas_combinado(proporcion_tono_por_tema)


In [ ]:
ruta_salida = "EstudioPoliticoApp/datasets/politicos_etiquetado_final.xlsx"

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    df_metadata.to_excel(writer, sheet_name="Metadata", index=False)
    df_posts.to_excel(writer, sheet_name="Posts", index=False)
    df_comentarios.to_excel(writer, sheet_name="Comentarios", index=False)
